<div align="right">
Massimo Nocentini<br>
<br>May 30, 2016: Balanced parens, Steep parallelograms
<br>May 29, 2016: Plane trees, Blocks
<br>May 28, 2016: Sigmoids, Linearization
<br>May 27, 2016: Binary and Ternary Trees, Dick paths
<br>May 26, 2016: structure generation
</div>
<br>
<div align="center">
<b>Abstract</b><br>
This document collect some examples about <i>recursively-defined structures</i>.
</div>

In [1]:
%matplotlib inline
%run ../python-libs/bits.py
%run ../python-libs/timing.py
%run ../python-libs/graycodes.py
%run ../python-libs/symbols.py

In [258]:
from collections import namedtuple
from contextlib import contextmanager

Anchor = namedtuple('Anchor', ['symbol', 'stars'])
Star = namedtuple('Star', ['row', 'col', 'offsets', 'link', 'symbol'])
Structure = namedtuple('Structure', ['anchors', 'stars', 'shapes', 'tables'])

# numbering and coordinates are inverses the one of the other                  
def coordinates(group):
    
    def do(anchor):
        c, r = divmod(anchor, group)
        return r, c # simply flip them

    return do

def numbering(group):
    
    def do(coordinate):
        r, c = coordinate
        return r + group*c

    return do

#_______________________________________________________________

@contextmanager
def extract_star(stars, structure, locating):
    
    coordinates, numbering = locating
    
    anchor, stars = low_bit_and_clear(stars)
    r, c = coordinates(anchor)
    
    anchors_table, stars_table = structure.tables
    del stars_table[r, c]

    # get the corresponding `shape` object
    s = structure.shapes[r, c]
    del structure.shapes[r, c]
    
    # and "place" it, namely cell at position (r, c) will be occupied by a new anchor object.
    anchor_tuple = s(r, c)

    current_anchors_table, current_stars_table = dict(anchors_table), dict(stars_table)

    # remember the anchor symbol
    current_anchors_table[r, c] = anchor_tuple.symbol
    
    yield (anchor, stars), anchor_tuple, (current_anchors_table, current_stars_table)


#_______________________________________________________________

def recursive_structures(shapes_spec, start_cell, locating):
    "Returns a generator of structures according `shapes` recursion schemas, starting at anchor `init`."
    
    from collections import defaultdict, deque
    
    shapes_descr, initial_shape = shapes_spec
    coordinates, numbering = locating
            
    def gen(generation):

        yield generation

        next_generation = []
        
        for structure in generation:
            
            anchors, stars, shapes, (anchors_table, stars_table) = structure
            
            while stars: # `stars` is an integer to be used as bitmask: 1 in position k 
                         # means that a ★ is present at cell numbered k
                
                with extract_star(stars, structure, locating) as (
                    (anchor, stars), anchor_tuple, (current_anchors_table, current_stars_table)):

                    # preparing for looping on stars and collecting new stars positions
                    augmented_stars, new_shapes = stars, dict(shapes)

                    for star in anchor_tuple.stars:
                        rs, cs, offsets = star.row, star.col, star.offsets

                        if (rs, cs) in anchors_table:
                            pass
                            #raise ValueError("Attempt to place a star in a cell already occupied by an anchor.")
                        
                        if offsets:
                            r_offset, c_offset = offsets
                            translating_stars = ones_of(augmented_stars)
                            augmented_stars = 0
                            inner_stars_table = {}
                            inner_shapes = {}
                            for ts in translating_stars:
                                ts_r, ts_c = coordinates(ts)
                                symbol = current_stars_table[ts_r, ts_c]
                                shape = new_shapes[ts_r, ts_c]
                                translated = ts_r + r_offset, ts_c + c_offset
                                augmented_stars = set_bit(augmented_stars, numbering(translated))
                                inner_stars_table[translated] = symbol
                                inner_shapes[translated] = shape
                                
                            current_stars_table = inner_stars_table
                            new_shapes = inner_shapes

                        current_stars_table[rs, cs] = star.symbol
                        new_shapes[rs, cs] = shapes_descr[star.link]
                        star_cell = numbering((rs,cs),)
                        augmented_stars = set_bit(augmented_stars, star_cell)


                    next_generation.append(Structure(anchors=set_bit(anchors, anchor), 
                                                     stars=augmented_stars, 
                                                     shapes=new_shapes, 
                                                     tables=(current_anchors_table, current_stars_table)))

        yield from gen(next_generation)
                
            
    initial_structure = Structure(anchors=0b0, 
                                  stars=1 << start_cell, 
                                  shapes={coordinates(start_cell):shapes_descr[initial_shape]}, 
                                  tables=({}, {coordinates(start_cell):'★'}))
    
    return gen([initial_structure])

#________________________________________________________________________________________
def make_pretty(dim, joiner='', separator=',', empty=' '):
    
    def pretty(structures,):
        from collections import defaultdict

        rows, cols = dim

        strings = []
        for anchors, stars, shapes, (anchors_table, stars_table) in structures:

            table = defaultdict(lambda: empty)
            
            for k, v in anchors_table.items():
                table[k] = table[k] + separator + v if k in table else v

            for k, v in stars_table.items():
                table[k] = table[k] + separator + v if k in table else v

            s = ''
            for r in range(rows):
                s += joiner.join(table[r, c] for c in range(cols)) + '\n'

            strings.append(s)

        return  ''.join(strings)
    
    return pretty

# Triangulated polygons

In [173]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
★▲★  :nord

★▲   :sw
 ★
 
▲★   :se
★

 ★
★▼   :nw

★
▼★   :ne

★▼★  :south

"""
shapes = {
    'north': lambda r, c: Anchor(symbol='▲', stars=[
                Star(row=r, col=c-1, offsets=None, link='nw', symbol='★'),
                Star(row=r, col=c+1, offsets=None, link='ne', symbol='★'),]),
    'sw':   lambda r, c: Anchor(symbol='▲', stars=[
                Star(row=r, col=c-1, offsets=None, link='nw', symbol='★'),
                Star(row=r+1, col=c, offsets=None, link='south', symbol='★'),]),
    'se':   lambda r, c: Anchor(symbol='▲', stars=[
                Star(row=r+1, col=c, offsets=None, link='south', symbol='★'),
                Star(row=r, col=c+1, offsets=None, link='ne', symbol='★'),]),
    'nw':   lambda r, c: Anchor(symbol='▼', stars=[
                Star(row=r, col=c-1, offsets=None, link='sw', symbol='★'),
                Star(row=r-1, col=c, offsets=None, link='north', symbol='★'),]),
    'ne':   lambda r, c: Anchor(symbol='▼', stars=[
                Star(row=r-1, col=c, offsets=None, link='north', symbol='★'),
                Star(row=r, col=c+1, offsets=None, link='se', symbol='★'),]),
    'south': lambda r, c: Anchor(symbol='▼', stars=[
                Star(row=r, col=c-1, offsets=None, link='sw', symbol='★'),
                Star(row=r, col=c+1, offsets=None, link='se', symbol='★'),]),
}

In [174]:
# skip this, now the above pretty printer should work
def make_pretty_old(dim, joiner=' ', separator=','):
    
    from collections import defaultdict

    rows, cols = dim
    
    def pretty(structures):
        strings = []
        for anchors, stars, shapes, (anchors_table, stars_table) in structures:
            #print(anchors_table)
            table = defaultdict(lambda: joiner)

            for a in ones_of(anchors):
                ca = coor(a)
                table[ca] = table[ca] + separator + anchors_table[ca] if ca in table else anchors_table[ca]

            for s in ones_of(stars):
                cs = coor(s)
                table[cs] = table[cs] + separator if cs in table else '★'

            s = ''
            for r in range(rows):
                s += joiner.join(table[r, c] for c in range(cols)) + '\n'

            strings.append(s)

        return  ''.join(strings)
    
    return pretty

In [175]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
bin_trees = recursive_structures((shapes, 'north'), rows*12+4, (coor, number))
representations = map(make_pretty((rows, cols),), bin_trees)

First generation contains only the *seed* symbol ★

In [176]:
print(next(representations))

                         
                         
                         
                         
            ★            
                         
                         
                         



Next generation:

In [177]:
print(next(representations))

                         
                         
                         
                         
           ★▲★           
                         
                         
                         



Next generation:

In [178]:
print(next(representations))

                         
                         
                         
           ★             
          ★▼▲★           
                         
                         
                         
                         
                         
                         
             ★           
            ▲▼★          
                         
                         
                         



Next generation:

In [179]:
print(next(representations))

                         
                         
                         
           ★             
         ★▲▼▲★           
          ★              
                         
                         
                         
                         
                         
          ★▲★            
           ▼▲★           
                         
                         
                         
                         
                         
                         
             ★           
           ▼▲▼★          
                         
                         
                         
                         
                         
                         
            ★▲★          
            ▲▼★          
                         
                         
                         
                         
                         
                         
                         
            ▲▼▲★         
              ★          
            

Next generation:

In [180]:
print(next(representations))

                         
                         
                         
         ★ ★             
        ★▼▲▼▲★           
          ★              
                         
                         
                         
                         
                         
           ★             
          ▲▼▲★           
         ★▼★             
                         
                         
                         
                         
                         
          ★▲★            
          ▲▼▲★           
                         
                         
                         
                         
                         
                         
             ★           
          ▲▼▲▼★          
                         
                         
                         
                         
                         
          ★              
         ★▼▲★            
           ▼▲★           
                         
            

# Binary trees

In [314]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ●
★ o ★  
"""
shapes = {
    'bintree': lambda r, c: Anchor(symbol='●', stars=[
                Star(row=r+1, col=c-1, offsets=None, link='bintree', symbol='★'),
                Star(row=r+1, col=c+1, offsets=None, link='bintree', symbol='★'),]),
    }

In [321]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
bin_trees = recursive_structures((shapes, 'bintree'), rows*12, (coor, number))
representations = map(make_pretty((rows, cols),), bin_trees)

First generation contains only the *seed* symbol ★

In [322]:
print(next(representations))

            ★            
                         
                         
                         
                         
                         
                         
                         



Next generation:

In [323]:
print(next(representations))

            ●            
           ★ ★           
                         
                         
                         
                         
                         
                         



Next generation:

In [324]:
print(next(representations))

            ●            
           ● ★           
          ★ ★            
                         
                         
                         
                         
                         
            ●            
             ●           
            ★ ★          
                         
                         
                         
                         
                         



Next generation:

In [325]:
print(next(representations))

            ●            
           ● ★           
          ● ★            
         ★ ★             
                         
                         
                         
                         
            ●            
           ● ★           
            ●            
           ★ ★           
                         
                         
                         
                         
            ●            
           ● ●           
            ★ ★          
                         
                         
                         
                         
                         
            ●            
             ●           
            ● ★          
           ★ ★           
                         
                         
                         
                         
            ●            
             ●           
              ●          
             ★ ★         
                         
                         
            

Next generation:

In [326]:
print(next(representations))

            ●            
           ● ★           
          ● ★            
         ● ★             
        ★ ★              
                         
                         
                         
            ●            
           ● ★           
          ● ★            
           ●             
          ★ ★            
                         
                         
                         
            ●            
           ● ★           
          ● ●            
           ★ ★           
                         
                         
                         
                         
            ●            
           ● ●           
          ● ★ ★          
                         
                         
                         
                         
                         
            ●            
           ● ★           
            ●            
           ● ★           
          ★ ★            
                         
            

# Ternary trees

In [188]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ●
★ ★ ★  
"""
threetree_shape = lambda r, c: [((r+1, c-1), (r+1, c), (r+1, c+1))]
shapes = {
    'threetree': lambda r, c: Anchor(symbol='●', stars=[
                Star(row=r+1, col=c-1, offsets=None, link='threetree', symbol='★'),
                Star(row=r+1, col=c, offsets=None, link='threetree', symbol='★'),
                Star(row=r+1, col=c+1, offsets=None, link='threetree', symbol='★'),]),
    }

In [189]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
three_trees = recursive_structures((shapes, 'threetree'), rows*12, (coor, number))
representations = map(make_pretty((rows, cols),), three_trees)

First generation contains only the *seed* symbol ★

In [190]:
print(next(representations))

            ★            
                         
                         
                         
                         
                         
                         
                         



Next generation:

In [191]:
print(next(representations))

            ●            
           ★★★           
                         
                         
                         
                         
                         
                         



Next generation:

In [192]:
print(next(representations))

            ●            
           ●★★           
          ★★★            
                         
                         
                         
                         
                         
            ●            
            ●★           
           ★★★           
                         
                         
                         
                         
                         
            ●            
             ●           
            ★★★          
                         
                         
                         
                         
                         



Next generation:

In [193]:
print(next(representations))

            ●            
           ●★★           
          ●★★            
         ★★★             
                         
                         
                         
                         
            ●            
           ●★★           
           ●★            
          ★★★            
                         
                         
                         
                         
            ●            
           ●●★           
           ★★★           
                         
                         
                         
                         
                         
            ●            
           ● ★           
            ●            
           ★★★           
                         
                         
                         
                         
            ●            
           ● ●           
            ★★★          
                         
                         
                         
            

# Dick paths

In [194]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ★
( o ★  
"""
shapes = {
    'dick': lambda r, c: Anchor(symbol='(', stars=[
                Star(row=r-1, col=c+1, offsets=(0, 2), link='dick', symbol='★'),
                Star(row=r, col=c+2, offsets=None, link='dick', symbol='★'),]),
    }

In [195]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
dick_paths = recursive_structures((shapes, 'dick'), rows-1, (coor, number))
representations = map(make_pretty((rows, cols),), dick_paths)

First generation contains only the *seed* symbol ★

In [196]:
print(next(representations))

                         
                         
                         
                         
                         
                         
                         
★                        



Next generation:

In [197]:
print(next(representations))

                         
                         
                         
                         
                         
                         
 ★                       
( ★                      



Next generation:

In [198]:
print(next(representations))

                         
                         
                         
                         
                         
  ★                      
 ( ★                     
(   ★                    
                         
                         
                         
                         
                         
                         
   ★                     
( ( ★                    



Next generation:

In [199]:
print(next(representations))

                         
                         
                         
                         
   ★                     
  ( ★                    
 (   ★                   
(     ★                  
                         
                         
                         
                         
                         
    ★                    
 ( ( ★                   
(     ★                  
                         
                         
                         
                         
                         
                         
 (   ★                   
(   ( ★                  
                         
                         
                         
                         
                         
    ★                    
   ( ★                   
( (   ★                  
                         
                         
                         
                         
                         
                         
     ★      

Next generation:

In [200]:
print(next(representations))

                         
                         
                         
    ★                    
   ( ★                   
  (   ★                  
 (     ★                 
(       ★                
                         
                         
                         
                         
     ★                   
  ( ( ★                  
 (     ★                 
(       ★                
                         
                         
                         
                         
                         
  (   ★                  
 (   ( ★                 
(       ★                
                         
                         
                         
                         
                         
  (                      
 (     ★                 
(     ( ★                
                         
                         
                         
                         
     ★                   
    ( ★                  
 ( (   ★    

# Sigmoids

In [201]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ★
― o o
    ★
"""
shapes = {
    'sigmoid': lambda r, c: Anchor(symbol='―', stars=[
                Star(row=r-1, col=c+1, offsets=None, link='sigmoid', symbol='★'),
                Star(row=r+1, col=c+2, offsets=None, link='sigmoid', symbol='★'),]),
    }

In [202]:
rows, cols = 9, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
sigmoids = recursive_structures((shapes, 'sigmoid'), 4, (coor, number))
representations = map(make_pretty((rows, cols),), sigmoids)

First generation contains only the *seed* symbol ★

In [203]:
print(next(representations))

                         
                         
                         
                         
★                        
                         
                         
                         
                         



Next generation:

In [204]:
print(next(representations))

                         
                         
                         
 ★                       
―                        
  ★                      
                         
                         
                         



Next generation:

In [205]:
print(next(representations))

                         
                         
  ★                      
 ―                       
―  ★                     
  ★                      
                         
                         
                         
                         
                         
                         
                         
―  ★                     
  ―                      
    ★                    
                         
                         



Next generation:

In [206]:
print(next(representations))

                         
   ★                     
  ―                      
 ―  ★                    
―  ★                     
  ★                      
                         
                         
                         
                         
                         
                         
 ―                       
―  ★                     
  ―                      
    ★                    
                         
                         
                         
                         
                         
 ―  ★                    
―  ―                     
     ★                   
                         
                         
                         
                         
                         
                         
    ★                    
―  ―                     
  ―  ★                   
    ★                    
                         
                         
                         
                         
            

Next generation:

In [207]:
print(next(representations))

                         
   ★                     
  ―                      
 ―  ★                    
―  ★                     
  ―                      
    ★                    
                         
                         
    ★                    
   ―                     
  ―  ★                   
 ―  ★                    
―  ★                     
                         
                         
                         
                         
                         
                         
  ―                      
 ―  ★                    
―  ―                     
     ★                   
                         
                         
                         
                         
                         
  ―  ★                   
 ―  ―                    
―     ★                  
                         
                         
                         
                         
                         
                         
            

# Linearization

In [208]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

""" 
● ★ ★  
"""
shapes = {
    'linear': lambda r, c: Anchor(symbol='●', stars=[
                Star(row=r, col=c+1, offsets=(0,2), link='linear', symbol='★'),
                Star(row=r, col=c+2, offsets=None, link='linear', symbol='★'),]),
    }

In [209]:
rows, cols = 3, 25
coor, number = coordinates(rows), numbering(rows)

linears = recursive_structures((shapes, 'linear'), 1, (coor, number))
representations = map(make_pretty((rows, cols),), linears)

First generation contains only the *seed* symbol ★

In [210]:
print(next(representations))

                         
★                        
                         



Next generation:

In [211]:
print(next(representations))

                         
●★★                      
                         



Next generation:

In [212]:
print(next(representations))

                         
●●★★★                    
                         
                         
● ●★★                    
                         



Next generation:

In [213]:
print(next(representations))

                         
●●●★★★★                  
                         
                         
●● ●★★★                  
                         
                         
●●  ●★★                  
                         
                         
● ●●★★★                  
                         
                         
● ● ●★★                  
                         



Next generation:

In [214]:
print(next(representations))

                         
●●●●★★★★★                
                         
                         
●●● ●★★★★                
                         
                         
●●●  ●★★★                
                         
                         
●●●   ●★★                
                         
                         
●● ●●★★★★                
                         
                         
●● ● ●★★★                
                         
                         
●● ●  ●★★                
                         
                         
●●  ●●★★★                
                         
                         
●●  ● ●★★                
                         
                         
● ●●●★★★★                
                         
                         
● ●● ●★★★                
                         
                         
● ●●  ●★★                
                         
                         
● ● ●●★★★                
            

# Balanced parens

In [215]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

""" 
( ★ ) ★  
"""
shapes = {
    'parens': lambda r, c: Anchor(symbol='(', stars=[
                Star(row=r, col=c+1, offsets=(0,3), link='parens', symbol='★'),
                Star(row=r, col=c+3, offsets=None, link='parens', symbol='★'),]),
    }

In [221]:
rows, cols = 3, 32
coor, number = coordinates(rows), numbering(rows)

parens = recursive_structures((shapes, 'parens'), 1, (coor, number))
representations = map(make_pretty((rows, cols),), parens)

First generation contains only the *seed* symbol ★

In [222]:
print(next(representations))

                                
★                               
                                



Next generation:

In [223]:
print(next(representations))

                                
(★ ★                            
                                



Next generation:

In [224]:
print(next(representations))

                                
((★ ★ ★                         
                                
                                
(  (★ ★                         
                                



Next generation:

In [225]:
print(next(representations))

                                
(((★ ★ ★ ★                      
                                
                                
((  (★ ★ ★                      
                                
                                
((    (★ ★                      
                                
                                
(  ((★ ★ ★                      
                                
                                
(  (  (★ ★                      
                                



Next generation:

In [226]:
print(next(representations))

                                
((((★ ★ ★ ★ ★                   
                                
                                
(((  (★ ★ ★ ★                   
                                
                                
(((    (★ ★ ★                   
                                
                                
(((      (★ ★                   
                                
                                
((  ((★ ★ ★ ★                   
                                
                                
((  (  (★ ★ ★                   
                                
                                
((  (    (★ ★                   
                                
                                
((    ((★ ★ ★                   
                                
                                
((    (  (★ ★                   
                                
                                
(  (((★ ★ ★ ★                   
                                
          

# Splitters

In [300]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
★
─  :horizontal
★

★ │ ★   :vertical
 
"""
shapes = {
    'horizontal': lambda r, c: Anchor(symbol='─', stars=[
                Star(row=r-1, col=c, offsets=None, link='vertical', symbol='★'),
                Star(row=r+1, col=c, offsets=None, link='vertical', symbol='★'),]),
    'vertical': lambda r, c: Anchor(symbol='│', stars=[
                Star(row=r, col=c-1, offsets=None, link='horizontal', symbol='★'),
                Star(row=r, col=c+1, offsets=None, link='horizontal', symbol='★'),]),
}

In [307]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
fractals = recursive_structures((shapes, 'vertical'), rows*12+4, (coor, number))
representations = map(make_pretty((rows, cols),), fractals)

First generation contains only the *seed* symbol ★

In [308]:
print(next(representations))

                         
                         
                         
                         
            ★            
                         
                         
                         



Next generation:

In [309]:
print(next(representations))

                         
                         
                         
                         
           ★│★           
                         
                         
                         



Next generation:

In [310]:
print(next(representations))

                         
                         
                         
           ★             
           ─│★           
           ★             
                         
                         
                         
                         
                         
             ★           
            │─           
             ★           
                         
                         



Next generation:

In [311]:
print(next(representations))

                         
                         
                         
          ★│★            
           ─│★           
           ★             
                         
                         
                         
                         
                         
                         
           ─│★           
          ★│★            
                         
                         
                         
                         
                         
             ★           
           ─│─           
             ★           
                         
                         
                         
                         
                         
            ★│★          
            │─           
             ★           
                         
                         
                         
                         
                         
                         
            │─           
            ★│★          
            

Next generation:

In [312]:
print(next(representations))

                         
                         
          ★              
          ─│★            
          ★─│★           
           ★             
                         
                         
                         
                         
                         
           │★            
           ─│★           
          ★│★            
                         
                         
                         
                         
            ★            
           │─            
           ─│,★★           
                         
                         
                         
                         
                         
                         
           │ ★           
           ─│─           
             ★           
                         
                         
                         
                         
                         
                         
          ★─│★           
          ─│★            
          

# Plane trees

In [266]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

""" 
★
Λ
 ★  
"""
shapes = {
    'plane': lambda r, c: Anchor(symbol='Λ', stars=[
                Star(row=r-1, col=c, offsets=None, link='plane', symbol='★'),
                Star(row=r+1, col=c+1, offsets=None, link='plane', symbol='★'),]),
    }

In [267]:
rows, cols = 10, 25
coor, number = coordinates(rows), numbering(rows)

plane_trees = recursive_structures((shapes, 'plane'), 4, (coor, number))
representations = map(make_pretty((rows, cols),), plane_trees)

First generation contains only the *seed* symbol ★

In [268]:
print(next(representations))

                         
                         
                         
                         
★                        
                         
                         
                         
                         
                         



Next generation:

In [269]:
print(next(representations))

                         
                         
                         
★                        
Λ                        
 ★                       
                         
                         
                         
                         



Next generation:

In [270]:
print(next(representations))

                         
                         
★                        
Λ                        
Λ★                       
 ★                       
                         
                         
                         
                         
                         
                         
                         
                         
Λ★                       
 Λ                       
  ★                      
                         
                         
                         



Next generation:

In [271]:
print(next(representations))

                         
★                        
Λ                        
Λ★                       
Λ★                       
 ★                       
                         
                         
                         
                         
                         
                         
                         
Λ★                       
ΛΛ                       
 ★★                      
                         
                         
                         
                         
                         
                         
                         
Λ                        
Λ★                       
 Λ                       
  ★                      
                         
                         
                         
                         
                         
                         
 ★                       
ΛΛ                       
 Λ★                      
  ★                      
                         
            

Next generation:

In [272]:
print(next(representations))

★                        
Λ                        
Λ★                       
Λ★                       
Λ★                       
 ★                       
                         
                         
                         
                         
                         
                         
Λ★                       
ΛΛ                       
Λ★★                      
 ★                       
                         
                         
                         
                         
                         
                         
Λ                        
Λ★                       
ΛΛ                       
 ★★                      
                         
                         
                         
                         
                         
                         
Λ                        
Λ                        
Λ★                       
 Λ                       
  ★                      
                         
            

# Blocks

In [259]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

""" 
★
▢ ★ 
"""
shapes = {
    'block': lambda r, c: Anchor(symbol='▢', stars=[
                Star(row=r-1, col=c, offsets=None, link='block', symbol='★'),
                Star(row=r, col=c+1, offsets=None, link='block', symbol='★'),]),
    }

In [260]:
rows, cols = 10, 25
coor, number = coordinates(rows), numbering(rows)

blocks = recursive_structures((shapes, 'block'), 4, (coor, number))
representations = map(make_pretty((rows, cols), joiner=' '), blocks)

First generation contains only the *seed* symbol ★

In [261]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
★                                                
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [262]:
print(next(representations))

                                                 
                                                 
                                                 
★                                                
▢ ★                                              
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [263]:
print(next(representations))

                                                 
                                                 
★                                                
▢ ★                                              
▢ ★                                              
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
  ★                                              
▢ ▢ ★                                            
                                                 
                                                 
                                                 
                                                 
                                                 


Next generation:

In [264]:
print(next(representations))

                                                 
★                                                
▢ ★                                              
▢ ★                                              
▢ ★                                              
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
  ★                                              
▢ ▢ ★                                            
▢ ★                                              
                                                 
                                                 
                                                 
                                                 
                                                 


Next generation:

In [265]:
print(next(representations))

★                                                
▢ ★                                              
▢ ★                                              
▢ ★                                              
▢ ★                                              
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
  ★                                              
▢ ▢ ★                                            
▢ ★                                              
▢ ★                                              
                                                 
                                                 
                                                 
                                                 
                                                 


# Steep Parallelograms

In [285]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

""" 
☆
▢ 

☆
▢ ★ 
"""
shapes = {
    'one_star': lambda r, c: Anchor(symbol='▢', stars=[
                Star(row=r-1, col=c, offsets=None, link='two_stars', symbol='☆'),]),
    'two_stars': lambda r, c: Anchor(symbol='▢', stars=[
                Star(row=r-1, col=c, offsets=None, link='two_stars', symbol='☆'),
                Star(row=r, col=c+1, offsets=None, link='one_star', symbol='★'),]),

    }

In [293]:
rows, cols = 14, 25
coor, number = coordinates(rows), numbering(rows)

steep_parallelograms = recursive_structures((shapes, 'one_star'), 7, (coor, number))
representations = map(make_pretty((rows, cols), joiner=' '), steep_parallelograms)

First generation contains only the *seed* symbol ★

In [294]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
★                                                
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [295]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
☆                                                
▢                                                
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [296]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                                                 
☆                                                
▢ ★                                              
▢                                                
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [297]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
☆                                                
▢ ★                                              
▢ ★                                              
▢                                                
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
  ☆                                              


Next generation:

In [298]:
print(next(representations))

                                                 
                                                 
                                                 
☆                                                
▢ ★                                              
▢ ★                                              
▢ ★                                              
▢                                                
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
  ☆                                              
▢ ▢                                              


Next generation:

In [299]:
print(next(representations))

                                                 
                                                 
☆                                                
▢ ★                                              
▢ ★                                              
▢ ★                                              
▢ ★                                              
▢                                                
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
  ☆                                              
▢ ▢                                              
▢ ★                                              


---
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Recursive Structures tutorial</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="massimo.nocentini@unifi.it" property="cc:attributionName" rel="cc:attributionURL">Massimo Nocentini</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/massimo-nocentini/competitive-programming/blob/master/tutorials/recursive-structures.ipynb" rel="dct:source">https://github.com/massimo-nocentini/competitive-programming/blob/master/tutorials/recursive-structures.ipynb</a>.